# Bootstrap2.5d Colab

Before running the script, go to the toolbar above and select:

    Runtime → Change Runtime type → Hardware accelerator → GPU

After the runtime has been changed, the necessary dependencies need to be downloaded.

In [ ]:
# Install dependencies
!pip install snakemake tqdm torch simpleitk albumentations -U -q

In [ ]:
#reload modules whenever they are updated
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import numpy as np
import SimpleITK as sitk
from glob import glob
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

Next, the bootstrap2.5d directory will be downloaded and the directories for the model's inputs and outputs will be created. 

In [ ]:
# Designate path of bootstrap2.5d directory
# Change to /content/drive/My Drive/bootstrap2.5d if downloading to Google Drive
boot_dir = "/content/bootstrap2.5d"

# Make the boot_dir and clone the GitHub repo
os.mkdir(boot_dir)
!git clone --single-branch --branch colab https://github.com/volume-em/bootstrap2.5d.git {boot_dir}

In [ ]:
# Create data directories
os.mkdir(os.path.join(boot_dir, 'models'))
os.mkdir(os.path.join(boot_dir, 'data'))
os.mkdir(os.path.join(boot_dir, 'data/train'))
os.mkdir(os.path.join(boot_dir, 'data/train/images'))
os.mkdir(os.path.join(boot_dir, 'data/train/masks'))
os.mkdir(os.path.join(boot_dir, 'data/target'))
os.mkdir(os.path.join(boot_dir, 'data/target/images'))

#this is optional, only applies if there is a ground truth mask for the target images
os.mkdir(os.path.join(boot_dir, 'data/target/masks'))

The repository has been cloned and the necessary directories have been created, now datasets can be added. To upload data, click the folder icon to the left. From there files can be added by:

1. Dragging files from your local file browser into colab
2. Right clicking on a folder in colab and pressing upload
3. Pressing "Mount Drive" at the top of the files tab. This will allow you to access files from your Google Drive in Colab. Files uploaded to Colab runtimes are deleted when sessions end, for permanent storage the bootstrap2.5d repository should be downloaded directly to your Google Drive. Alternatively, if the repo is downloaded to the runtime's temporary storage one can click and drag files into Drive.



## Example

In this example, we are downloading data from the paper [Automatic segmentation of mitochondria and endolysosomes in volumetric electron microscopy data](https://www.sciencedirect.com/science/article/abs/pii/S0010482520300792?via%3Dihub). This dataset contains training volumes with segmented mitochondria and lysosomes. Because the organelles are segmented in separate files, we have to combine them in the cell below.

In [ ]:
# Run the setup script, save data in the data folder
data_dir = os.path.join(boot_dir, 'data')
setup_script = os.path.join(boot_dir, 'example_data/setup_data.py')
!python {setup_script} {data_dir}

Now that the dataset has been downloaded, we can visualize what our training volumes and masks look like.

In [ ]:
# Create a custom colormap so that background is transparent
# get colormap
ncolors = 256
color_array = plt.get_cmap('viridis')(range(ncolors))

# change alpha values
#color_array[:,-1] = np.linspace(0.0, 1.0, ncolors)
color_array[0, -1] = 0.0

# create a colormap object
map_object = LinearSegmentedColormap.from_list(name='viridis_alpha',colors=color_array)

# register this new colormap with matplotlib
plt.register_cmap(cmap=map_object)

In [ ]:
# Take some 2D slices of a 3D volume
vol_index = 0 #index of file in the train/images directory

#get a list of the train image volumes
impaths = np.sort(glob(os.path.join(data_dir, 'train/images/*.nii.gz')))
impath = impaths[vol_index]

#load the image and the mask and convert them to arrays
train_image = sitk.GetArrayFromImage(sitk.ReadImage(impath))
train_mask = sitk.GetArrayFromImage(sitk.ReadImage(impath.replace('/images/', '/masks/')))

#sample 12 evenly spaced slices from the xy plane
slice_indices = np.linspace(0, train_image.shape[0] - 1, num=12, dtype='int')

#create subplots
f, ax = plt.subplots(2, 6, figsize=(16, 8))

#plot the images
c = 0
for y in range(2):
    for x in range(6):
        slice_index = slice_indices[c]
        ax[y, x].set_title(f'Slice {slice_index}')
        ax[y, x].imshow(train_image[slice_index], cmap='gray')
        ax[y, x].imshow(train_mask[slice_index], alpha=0.3, cmap='viridis_alpha')
        ax[y, x].set_yticks([])
        ax[y, x].set_xticks([])
        c += 1
        
plt.tight_layout()

Now we're ready to run the snakemake. The only config parameter that we need to pass is the number of classes in the segmentation mask. In this case we have 3: background, lysosomes, and mitochondria. For binary segmentation, we would set n_classes=1.

Other hyperparameters or file paths can be changed by directly editing the Snakefile. Text files can be edited directly in Colab by double clicking them in the file browser.

In [ ]:
snakefile = os.path.join(boot_dir, 'Snakefile')
!snakemake -s {snakefile} --cores all --config n_classes=3

Now that predictions have been made, we can visualize how the two algorithms performed.

In [ ]:
# Take some 2D slices of a 3D volume
vol_index = 0 #index of file in the target/images directory

#get a list of the train image volumes
impaths = np.sort(glob(os.path.join(data_dir, 'target/images/*.nii.gz')))
impath = impaths[vol_index]

#load the image and mask and predictions and convert them to arrays
target_image = sitk.GetArrayFromImage(sitk.ReadImage(impath))
target_mask = sitk.GetArrayFromImage(sitk.ReadImage(impath.replace('/images/', '/masks/')))
target_super_preds = sitk.GetArrayFromImage(sitk.ReadImage(impath.replace('/images/', '/super_preds/')))
target_weaksuper_preds = sitk.GetArrayFromImage(sitk.ReadImage(impath.replace('/images/', '/weaksuper_preds/')))

#sample 6 evenly spaced slices from the xy plane
slice_indices = np.linspace(0, train_image.shape[0] - 1, num=6, dtype='int')
column_names = ['Ground Truth', 'Step 1. Super Prediction', 'Step 2. Weak Super Prediction']

#create subplots
f, ax = plt.subplots(6, 3, figsize=(16, 20))

#plot the images
c = 0
for y in range(6):
    for x, overlay in enumerate([target_mask, target_super_preds, target_weaksuper_preds]):
        slice_index = slice_indices[c]
        ax[y, x].set_title(column_names[x])
        ax[y, x].set_ylabel(f'Slice {slice_index}')
        ax[y, x].imshow(target_image[slice_index], cmap='gray')
        ax[y, x].imshow(overlay[slice_index], alpha=0.3)
        ax[y, x].set_yticks([])
        ax[y, x].set_xticks([])
    c += 1
        
plt.tight_layout()